# Face Alignment - Model Training

- Add the project's root directory (two levels up) to the Python path so the modules can be imported, even if they arent in the current working directory:

In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join('..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

- Import the required libraries and modules, as well as our utility functions:

In [ ]:
import numpy as np
import joblib
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA

from src.utils import load_config, get_project_root

- Load the config using the utility function. Get paths to relevant folders/files needed to save and retrieve files:

In [ ]:
config = load_config()

processed_train_data_path = config['data']['task2']['processed']['train']
model_path = config['data']['task2']['models']

processed_train_data = os.path.join(get_project_root(), processed_train_data_path.replace('/', os.sep), "processed_face_alignment_train_images_features.npz")
selected_model_path = os.path.join(get_project_root(), model_path.replace('/', os.sep))

- Load the training data. Extract the features and points from it. This will allow us to train a model as we can train it and then validate it:

In [ ]:
train_data = np.load(processed_train_data, allow_pickle=True)

In [ ]:
X = train_data['features']
y = train_data['points'].reshape((train_data['points'].shape[0], -1))

- Split the data into a training and validation set. This uses a test size of 0.2, which means an 80/20 split between training and validation sets respectively:

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=12)

In [ ]:
pca = PCA(n_components=100)
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)

In [ ]:
pca_filename = "pca_transform.pkl"
pca_path = os.path.join(get_project_root(), model_path.replace('/', os.sep), pca_filename)
joblib.dump(pca, pca_path)

In [ ]:
models = {
    'Ridge Regression': Ridge(alpha=1.0),
    'Linear Regression': LinearRegression(),
    'SVR': MultiOutputRegressor(SVR(kernel='linear', C=0.1, max_iter=1000)),
    'Random Forest': RandomForestRegressor(n_estimators=50, max_depth=10, max_features='sqrt', random_state=42)
}

In [ ]:
results = []
best_model = None
lowest_mse = float('inf')

In [ ]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    mse = mean_squared_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)

    results.append({'Model': name, 'MSE': mse, 'R2': r2})

    print(f"\n{name} Evaluation:")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R2 Score: {r2:.4f}")

    if mse < lowest_mse:
        best_model = model
        best_model_name = name
        lowest_mse = mse


In [ ]:
results_df = pd.DataFrame(results).sort_values(by="MSE")
ax = results_df.plot(x='Model', y=['MSE', 'R2'], kind='bar', figsize=(10, 5), legend=True)
plt.title('Model Performance (Sorted by MSE)')
plt.ylabel("Score")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
print(f"Best Model: {best_model}")

In [ ]:
filename = "best_model.pkl"

os.makedirs(selected_model_path, exist_ok=True)
    
full_path = os.path.join(selected_model_path, filename)

joblib.dump(best_model, full_path)
print(f"Model Saved: {full_path}")